In [30]:
# octane_combutsion.ipynb

# Cell 1

import pulp #pulp module is used for optimization


def init_prob(terms): #help function, tell it the number of unknowns
    # The goal is to minimize total atom count (POAC)
    # https://www.quantumstudy.com/chemistry/stoichiometry
    prob = pulp.LpProblem(sense=pulp.LpMinimize)
    #minimize sum of unknowns
    # Create decision variables (terms in chemical equation, +1 if ionic)
    # Each term coefficient must ultimately be an integer > 1
    x0, x1, x2, x3, x4, x5 = ((None),) * 6
    if terms >= 4: #declare coefficients
        x0 = pulp.LpVariable(name="x0", lowBound=1, cat="Integer")
        x1 = pulp.LpVariable(name="x1", lowBound=1, cat="Integer")
        x2 = pulp.LpVariable(name="x2", lowBound=1, cat="Integer")
        x3 = pulp.LpVariable(name="x3", lowBound=1, cat="Integer")
    if terms >= 5:
        x4 = pulp.LpVariable(name="x4", lowBound=1, cat="Integer")
    if terms >= 6:
        x5 = pulp.LpVariable(name="x5", lowBound=1, cat="Integer")

    # Define objective function based upon number of terms
    if terms == 4:
        prob += x0 + x1 + x2 + x3 #adding decision variables to prob
        return prob, x0, x1, x2, x3
    elif terms == 5:
        prob += x0 + x1 + x2 + x3 + x4
        return prob, x0, x1, x2, x3, x4
    elif terms == 6:
        prob += x0 + x1 + x2 + x3 + x4 + x5
        return prob, x0, x1, x2, x3, x4, x5


def solve_prob(prob, *x):
    status = prob.solve() #tell pulp to solve minimzation problem
    # Display if solution found is optimal, feasible, or infeasible
    print(pulp.LpStatus[status])

    # Display the final value of the decision variables
    if len(x[0]) >= 4:
        print(f"x0 = {int(pulp.value(x[0][0]))}")
        print(f"x1 = {int(pulp.value(x[0][1]))}")
        print(f"x2 = {int(pulp.value(x[0][2]))}")
        print(f"x3 = {int(pulp.value(x[0][3]))}")
    if len(x[0]) >= 5:
        print(f"x4 = {int(pulp.value(x[0][4]))}")
    if len(x[0]) >= 6:
        print(f"x5 = {int(pulp.value(x[0][5]))}")


# Equation 1: C8H18 + O2 -> CO2 + H2O 

prob, *x = init_prob(terms=4)
prob += 8* x[0] == x[2]  # Carbon (C)
prob += 2 * x[1] == 2 * x[2] + x[3]  # Oxygen (O)
prob += 18 * x[0] == 2 * x[3]  # Hydrogen (H)
print(solve_prob(prob, x))

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Dec  8 2020 

command line - cbc /tmp/47edf31b1721484e8da5f7c3bf3c4877-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/47edf31b1721484e8da5f7c3bf3c4877-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 28 RHS
At line 32 BOUNDS
At line 37 ENDATA
Problem MODEL has 3 rows, 4 columns and 7 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 30.5 - 0.00 seconds
Cgl0003I 0 fixed, 3 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 1 rows, 2 columns (2 integer (0 of which binary)) and 2 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 61 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 61

In [91]:
# Cell 2
from sympy import *
from fractions import Fraction

A = Matrix([[8, 0, -1, 0], [18, 0, 0, -2], [0, 2, -2, -1]])
#Each column in this matrix represents a molecule. For example, column 1 
# is octane, which has 8 carbons, 18 hydrogens, and 0 oxygens. The left two 
#columns (molecules) are positive, because they are on the left hand side of the 
#chemical equation. Right two columns (molecules) are on the RHS and are negative.

x = A.rref() #finds reduced echelon form
solns = list(x[0][3::4]) #these isolate the numbers (x0', x1', and x2') which 
#will become the coefficients x0, x1, and x2.
solns.append(int(-1)) #Have to add x3' manually (it's the free variable)

print(f'{solns} \n')

for i, item in enumerate(solns):
    item = item * -Fraction(min(solns)).denominator #turn xi' into xi, for 
    #i = 0, 1, 2, 3
    print(f'x{i} = {item}')

[-1/9, -25/18, -8/9, -1] 

x0 = 2
x1 = 25
x2 = 16
x3 = 18
